In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
from collections import defaultdict

def find_frequent_1_subgraphs(graphs, min_sup):
    subgraph_counts = defaultdict(int)
    for graph in graphs:
        for node in graph.nodes:
            subgraph_counts[frozenset([node])] += 1
    
    frequent_1_subgraphs = [list(subgraph) for subgraph, count in subgraph_counts.items() if count >= min_sup]
    return frequent_1_subgraphs

def find_frequent_2_subgraphs(graphs, min_sup):
    subgraph_counts = defaultdict(int)
    for graph in graphs:
        for edge in graph.edges:
            subgraph_counts[frozenset([edge])] += 1
    
    frequent_2_subgraphs = [list(subgraph) for subgraph, count in subgraph_counts.items() if count >= min_sup]
    return frequent_2_subgraphs

def count_support(subgraph, graphs):
    count = 0
    for graph in graphs:
        if contains_subgraph(graph, subgraph):
            count += 1
    return count

def contains_subgraph(graph, subgraph):
    subgraph_edges = set()
    for edge in subgraph:
        if not graph.has_edge(*edge):
            return False
        subgraph_edges.add(edge)
    
    return all(graph.has_edge(*edge) for edge in subgraph_edges)

def generate_candidates(F_prev, k):
    candidates = set()
    F_prev_list = list(F_prev)
    for i in range(len(F_prev_list)):
        for j in range(i + 1, len(F_prev_list)):
            sub1 = F_prev_list[i]
            sub2 = F_prev_list[j]
            candidate = frozenset(sub1 + sub2)
            if len(candidate) == k:
                candidates.add(candidate)
    return candidates

def prune_candidates(Ck, F_prev):
    pruned_candidates = set()
    for candidate in Ck:
        all_subgraphs_frequent = True
        for sub in generate_subgraphs(candidate):
            if sub not in F_prev:
                all_subgraphs_frequent = False
                break
        if all_subgraphs_frequent:
            pruned_candidates.add(candidate)
    return pruned_candidates

def generate_subgraphs(subgraph):
    subgraphs = set()
    subgraph_list = list(subgraph)
    for i in range(len(subgraph_list)):
        for j in range(i + 1, len(subgraph_list)):
            subgraphs.add(frozenset([subgraph_list[i], subgraph_list[j]]))
    return subgraphs

def fsg_algorithm(graphs, min_sup):
    # Initialization
    F1 = find_frequent_1_subgraphs(graphs, min_sup)
    F2 = find_frequent_2_subgraphs(graphs, min_sup)
    F = F1 + F2

    k = 3
    F_prev = F2

    while F_prev:
        # Generate candidate k-subgraphs
        Ck = generate_candidates(F_prev, k)
        
        # Prune candidates
        Ck = prune_candidates(Ck, F_prev)
        
        # Count support for candidates
        Fk = [list(c) for c in Ck if count_support(c, graphs) >= min_sup]
        
        # If no frequent k-subgraphs found, stop
        if not Fk:
            break
        
        # Update F
        F.extend(Fk)
        F_prev = Fk
        k += 1

    return F

def create_networkx_graph(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

def visualize_graphs(graphs, title='Graphs'):
    """
    Visualizes a list of graphs.
    Args:
        graphs (list of nx.Graph): List of networkx graphs to visualize.
        title (str): Title for the visualization.
    """
    num_graphs = len(graphs)
    fig, axes = plt.subplots(1, num_graphs, figsize=(15, 5))
    if num_graphs == 1:
        axes = [axes]

    for i, graph in enumerate(graphs):
        ax = axes[i]
        pos = nx.spring_layout(graph)  # Positions nodes using the Fruchterman-Reingold force-directed algorithm
        nx.draw(graph, pos, with_labels=True, ax=ax, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
        ax.set_title(f'Graph {i+1}')

    plt.suptitle(title)
    plt.show()

def visualize_frequent_subgraphs(graphs, frequent_subgraphs, title='Frequent Subgraphs'):
    """
    Visualizes the frequent subgraphs within the original graphs.
    Args:
        graphs (list of nx.Graph): List of networkx graphs where the subgraphs are found.
        frequent_subgraphs (list of list of tuples): List of frequent subgraphs, where each subgraph is a list of edges.
        title (str): Title for the visualization.
    """
    num_graphs = len(graphs)
    fig, axes = plt.subplots(1, num_graphs, figsize=(15, 5))
    if num_graphs == 1:
        axes = [axes]

    for i, graph in enumerate(graphs):
        ax = axes[i]
        pos = nx.spring_layout(graph)  # Positions nodes using the Fruchterman-Reingold force-directed algorithm
        nx.draw(graph, pos, with_labels=True, ax=ax, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
        
        # Highlight frequent subgraphs in this graph
        for subgraph in frequent_subgraphs:
            subgraph_edges = set(tuple(sorted(edge)) for edge in subgraph)  # Convert edges to sorted tuples
            graph_edges = set(tuple(sorted(edge)) for edge in graph.edges)
            if subgraph_edges.issubset(graph_edges):
                nx.draw_networkx_edges(graph, pos, edgelist=subgraph_edges, edge_color='red', width=2, ax=ax)

        ax.set_title(f'Graph {i+1}')

    plt.suptitle(title)
    plt.show()

# Define example graphs
graph_data = [
    [(1, 2), (2, 3)],
    [(1, 2), (2, 4), (2, 3)],
    [(1, 3), (3, 4)]
]

graphs = [create_networkx_graph(edges) for edges in graph_data]
visualize_graphs(graphs)
min_sup = 2
frequent_subgraphs = fsg_algorithm(graphs, min_sup)
#frequent_graphs = [create_networkx_graph(edges) for edges in frequent_subgraphs]
print(frequent_subgraphs)
